In [1]:
import pandas as pd
import numpy as np
import json
import folium

ModuleNotFoundError: No module named 'folium'

In [2]:
data = pd.read_pickle("data/data.pkl")
thematique = pd.read_pickle("data/Thématique.pkl")

In [3]:
thematique

,Votation,Année,Thématique
0,14.06.1981 Protection des consommateurs,1981,Economie
1,29.11.1981 Régime financier,1981,Finances publiques
2,14.06.1981 Egalité entre hommes et femmes,1981,Politique sociale
3,06.06.1982 Code pénal suisse,1982,Régime politique
4,06.06.1982 Loi sur les étrangers,1982,Politique sociale
5,28.11.1982 Surveillance des prix (initiative),1982,Economie
6,28.11.1982 Surveillance des prix (contre-projet),1982,Economie
7,04.12.1983 Naturalisation facilitée,1983,Régime politique
8,04.12.1983 Droit de la nationalité,1983,Régime politique
9,27.02.1983 Article sur l'énergie,1983,"Infrastructure, aménagement, environnement"


In [22]:
switzerland_coord = [46.765213, 8.252444]
town_geo_path = r'data/switzerland_borders/admin_level_8.geojson'
geo_json_data = json.load(open(town_geo_path, encoding="utf8"))
commune = [x['name']  for x in geo_json_data['features']]



In [24]:
data_t = data.merge(thematique , on = 'Votation')[['Thématique','Commune','Oui en %']]

data_t = data_t[data_t['Commune'].isin(commune)]
data_t.head()

,Thématique,Commune,Oui en %
0,Politique sociale,Aeugst am Albis,34.9
1,Politique sociale,Affoltern am Albis,30.3
2,Politique sociale,Bonstetten,34.3
3,Politique sociale,Hausen am Albis,35.5
4,Politique sociale,Hedingen,31.1


In [25]:

for theme, data_theme in data_t.groupby('Thématique') :
    data_theme = data_theme.groupby('Commune', as_index  = False).mean()
    map1 = folium.Map(location=switzerland_coord, zoom_start=8)
    map1.choropleth(geo_data = geo_json_data, \
                                    data = data_theme, \
                                    columns = ['Commune', 'Oui en %'], \
                                    key_on = 'feature.name', \
                                    fill_color = 'RdYlGr', \
                                    fill_opacity = 0.7, \
                                    line_opacity = 0.2, \
                                    legend_name = 'yes in % given to the theme ' + theme)
    
    map1.save('map/map_theme/map_'+theme+'.html')

NameError: name 'folium' is not defined

In [17]:
data_t_light

,Thématique,Commune,Oui en %
0,Politique sociale,Aeugst am Albis,34.9
1,Politique sociale,Affoltern am Albis,30.3
2,Politique sociale,Bonstetten,34.3
3,Politique sociale,Hausen am Albis,35.5
4,Politique sociale,Hedingen,31.1
5,Politique sociale,Kappel am Albis,21.2
6,Politique sociale,Knonau,31.0
7,Politique sociale,Maschwanden,24.0
8,Politique sociale,Mettmenstetten,33.0
9,Politique sociale,Obfelden,31.0
